# 计算反应能

In [1]:
from pymatgen.ext.matproj import MPRester
from pymatgen import Composition
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.core.units import FloatWithUnit
from pymatgen.analysis.reaction_calculator import ComputedReaction

In [2]:
a = MPRester()
all_entries = a.get_entries_in_chemsys(['Ca', 'C', 'O'])

In [3]:
all_entries[0]

ComputedEntry mp-10683 - Ca1
Energy = -1.6033
Correction = 0.0000
Parameters:
run_type = GGA
is_hubbard = False
pseudo_potential = {'functional': 'PBE', 'labels': ['Ca_sv'], 'pot_type': 'paw'}
hubbards = {}
potcar_symbols = ['PBE Ca_sv']
oxide_type = None
Data:
oxide_type = None

In [4]:
# 获取最低能量条目 composition.
def get_most_stable_entry(formula):
    relevant_entries = [entry for entry in all_entries 
                        if entry.composition.reduced_formula == Composition(formula).reduced_formula]
    relevant_entries = sorted(relevant_entries, key=lambda e: e.energy_per_atom) #按能量由低到高的方向对条目排序
    return relevant_entries[0]

In [5]:
CaO = get_most_stable_entry("CaO")
CaO

ComputedEntry mp-2605 - Ca1 O1
Energy = -12.8785
Correction = -0.7023
Parameters:
run_type = GGA
is_hubbard = False
pseudo_potential = {'functional': 'PBE', 'labels': ['O', 'Ca_sv'], 'pot_type': 'paw'}
hubbards = {}
potcar_symbols = ['PBE O', 'PBE Ca_sv']
oxide_type = oxide
Data:
oxide_type = oxide

In [6]:
CO2 = get_most_stable_entry("CO2")
CaCO3 = get_most_stable_entry("CaCO3")

In [7]:
reaction = ComputedReaction([CaO, CO2], [CaCO3])
reaction

CaO + CO2 -> CaCO3

In [8]:
energy = FloatWithUnit(reaction.calculated_reaction_energy, "eV atom^-1")
energy

-1.522750712500006

In [9]:
print("{}, {:.3f}kJ mol^-1(calc)".format(reaction, energy.to("kJ mol^-1")))

CaO + CO2 -> CaCO3, -146.923kJ mol^-1(calc)


In [12]:
# 获得实验数据
exp_CaO = a.get_exp_entry("CaO")
exp_CaCO3 = a.get_exp_entry("CaCO3")

In [13]:
# MP数据库不包含气相试验数据条目, 该值取自`NIST`, 然后手动创建条目, 单位应该为`kJ/mol`
exp_CO2 = ComputedEntry("CO2", -393.51)
exp_CO2

ComputedEntry None - C1 O2
Energy = -393.5100
Correction = 0.0000
Parameters:
Data:

In [14]:
exp_reaction = ComputedReaction([exp_CaO, exp_CO2], [exp_CaCO3])
exp_reaction

CaO + CO2 -> CaCO3

In [15]:
print("{}, {:.3f} kJ mol^-1(expe)".format(exp_reaction, exp_reaction.calculated_reaction_energy))

CaO + CO2 -> CaCO3, -178.300 kJ mol^-1(expe)
